In [ ]:
# === Step 1: Import Libraries ===
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scipy import signal
import seaborn as sns
from sklearn.metrics import confusion_matrix
import zipfile

# === Step 2: Mount Google Drive (if needed) ===


# Paths (Update If Needed)
zip_file = "/content/train.zip"  # Change this to the correct ZIP file path
extract_folder = "/content/trainimages"  # Folder where images will be extracted
csv_file = "/content/trainLabels.csv"  # Change if needed

# Extract ZIP File
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

with zipfile.ZipFile(zip_file, 'r') as archive:
    archive.extractall(extract_folder)

# Define the image folder
image_folder = "/content/trainimages/train" # Update based on extracted structure

# === Step 3: Load Dataset ===
df = pd.read_csv(csv_file)

# Function to load images
def load_image(image_id, img_size=(28, 28)):
    img_path = os.path.join(image_folder, f"{image_id}.png")  # Assuming PNG format
    if os.path.exists(img_path):
        img = imread(img_path)
        img_resized = resize(img, img_size)  # Resize to 28x28
        img_gray = rgb2gray(img_resized)  # Convert to grayscale
        return img_gray
    else:
        print(f"Image {image_id}.png not found!")
        return None

# === Step 4: Preprocess Data ===
X_data = []
y_data = []

for index, row in df.iterrows():
    img = load_image(row["id"])
    if img is not None:
        X_data.append(img)
        y_data.append(row["label"])

# Convert to NumPy arrays
X_data = np.array(X_data).reshape(len(X_data), 28, 28, 1)  # Add channel dimension
y_data = np.array(pd.factorize(y_data)[0])  # Convert labels to numbers

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# One-Hot Encoding Labels
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# === Step 5: Design CNN Model (A1) ===
model = Sequential([
    Conv2D(64, (3,3), input_shape=(28, 28, 1), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# === Step 6: Train CNN Model (A2) ===
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# === Step 7: Plot Training & Validation Loss (A3) ===
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

# === Step 8: Evaluate Model Accuracy (A4) ===
score = model.evaluate(X_test, y_test, verbose=1)
print('\nCNN Model Accuracy:', score[1])

# === Step 9: Inspect CNN Filters (A5) ===
filters, biases = model.layers[0].get_weights()

# Plot first 5 filters
for i in range(5):
    plt.imshow(filters[:,:,0,i], cmap='gray')
    plt.title(f"Filter {i+1}")
    plt.show()

# === Step 10: Inspect Filter Impact on an Image (A6) ===
sample_image = X_train[10].reshape(28, 28)
plt.imshow(sample_image, cmap='gray')
plt.title("Original Image")
plt.show()

# Apply filter to image
filtered_image = signal.convolve2d(sample_image, filters[:,:,0,1].reshape(3,3), boundary='symm', mode='same')
plt.imshow(filtered_image, cmap='gray')
plt.title("Filtered Image")
plt.show()

# === Step 11: Design & Train Fully Connected Model (A7) ===
fc_model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

fc_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

fc_history = fc_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# === Step 12: Plot Training vs Validation Loss (A8) ===
plt.plot(fc_history.history['loss'], label='Training Loss')
plt.plot(fc_history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Fully Connected Network - Training vs Validation Loss")
plt.show()

# === Step 13: Test & Evaluate Fully Connected Model (A8) ===
fc_score = fc_model.evaluate(X_test, y_test, verbose=1)
print('\nFully Connected Network Accuracy:', fc_score[1])
